In [1]:
import sys
print(sys.version)

3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]


For the sake of entertaining everyone, this is a basic stopwatch for this demo.

In [2]:
from contextlib import contextmanager
from time import sleep, time
from threading import active_count

initial_thread_count = active_count()

def additional_thread_count():
  thread_count = active_count() - initial_thread_count

  return thread_count if thread_count > 0 else 0

@contextmanager
def stopwatch(name: str):
  print(f'Stopwatch: {name}: Started: {additional_thread_count()} thread(s)')
  starting_time = time()
  yield
  print(f'Stopwatch: {name}: Stopped: {time() - starting_time:.3f}s, {additional_thread_count()} thread(s)')

Now, let's define a CPU-bound task.

In [3]:
def run_in_sync(id: str, delay: int):
  starting_time = time()
  while time() - starting_time < delay:
    sys.stdout.write(f'{id}')
    sleep(0.5)
    sys.stdout.write(f'{id}')
  sys.stdout.write(f'[{id}:stopped:{additional_thread_count()}t] ')
  return id

Suppose we have two runs.

In [4]:
runs = ['+', '-', '#']

Now, let's make two runs without multitasking.

In [5]:
with stopwatch('Set #1'):
  results = [
    run_in_sync(id, 5)
    for id in runs
  ]
  print(f'\nresults = {results}')

Stopwatch: Set #1: Started: 0 thread(s)
++++++++++++++++++++[+:stopped:0t] --------------------[-:stopped:0t] ####################[#:stopped:0t] 
results = ['+', '-', '#']
Stopwatch: Set #1: Stopped: 15.028s, 0 thread(s)


Well, this is a bit too long. Let's use multithreading to speed things up.
> For this demo, we use `ThreadPoolExecutor` for simplicity.

In [6]:
from concurrent.futures import as_completed, Future
from concurrent.futures.thread import ThreadPoolExecutor
from typing import Any, List, Dict

with stopwatch('Set #2'), ThreadPoolExecutor(max_workers=2) as pool:
  futures = [
    pool.submit(run_in_sync, id, 5)
    for id in runs
  ]

  results = [
    f.result()
    for f in as_completed(futures)
  ]

  print(f'\nresults = {results}')

Stopwatch: Set #2: Started: 0 thread(s)
+---++--++--++--++--++--+++--+--++--++-[-:stopped:2t] #+[+:stopped:2t] ###################[#:stopped:2t] 
results = ['-', '+', '#']
Stopwatch: Set #2: Stopped: 10.026s, 0 thread(s)


Now, let's try with `asyncio` but keep using `run_in_sync`.

In [7]:
import asyncio
from multiprocessing import Process

async def run_in_async(id: str, delay: int):
  starting_time = time()
  while time() - starting_time < delay:
    sys.stdout.write(f'{id}')
    await asyncio.sleep(0.5)
    sys.stdout.write(f'{id}')
  sys.stdout.write(f'[{id}:stopped:{additional_thread_count()}t] ')
  return id

async def demo_with_just_await(runs: List[str]):
  results = []
  for id in runs:
    results.append(await run_in_async(id, 5))
    # ↑ As you can see, while "await" will automatically create a task from a
    #   returning coroutine, "await" also block until the awaited task finishes.

  print(f'\nresults = {results}')

async def demo_with_explicit_task_creation(runs: List[str]):
  tasks = []
  for id in runs:
    coroutine = run_in_async(id, 5)  # ← This is just to get a coroutine for scheduling.
    task = asyncio.create_task(coroutine)  # ← This will schedule the coroutine right away.
    tasks.append(task)
  
  results = []
  for t in tasks:
    results.append(await t)

  print(f'\nresults = {results}')

def run_in_process():
  with stopwatch('Set #3'):
    asyncio.run(demo_with_just_await(runs))

  with stopwatch('Set #4'):
    asyncio.run(demo_with_explicit_task_creation(runs))

# Normally, we can just invoke asyncio.run(demo()). However, Jupyter's kernel
# seems to be using asyncio. So, we use a different thread to bypass the limit.
p = Process(target=run_in_process)
p.start()
p.join()


Stopwatch: Set #3: Started: 0 thread(s)
++++++++++++++++++++[+:stopped:0t] --------------------[-:stopped:0t] ####################[#:stopped:0t] 
results = ['+', '-', '#']
Stopwatch: Set #3: Stopped: 15.034s, 0 thread(s)
Stopwatch: Set #4: Started: 0 thread(s)
+-#++--##++--##++--##++--##++--##++--##++--##++--##++--##+[+:stopped:0t] -[-:stopped:0t] #[#:stopped:0t] 
results = ['+', '-', '#']
Stopwatch: Set #4: Stopped: 5.013s, 0 thread(s)


The difference between **multithreading (`threading`)** and **cooperative multitasking (`asyncio`)** can be summarized like this.

| | Multithreading (`threading`) | Cooperative Multitasking (`asyncio`) |
| --- | --- | --- |
| Method declaration | `def foo()` | `async def foo()` |
| Method Invocation: `def()` | Whatever is given in `return` | Corouting of `foo()` |
| Unit of Work | Thread | Task |
| Scheduler | Operating System | Event Loop |

---